In [ ]:
import os
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta import *
import re

import findspark

findspark.init()

warehouse_diretory_path = '[YOUR_WAREHOUSE]'

conf = SparkConf()
conf.setAll(
    [
        ('spark.master', 'local[*]'), 
        ('spark.driver.host', 'localhost'),
        ('spark.app.name', 'Illumina Human Methylation 450 Manifest Importer'),
        ('spark.ui.showConsoleProgress', 'true'),
        ('spark.sql.execution.arrow.pyspark.enabled', 'false'),                   
        ('spark.sql.execution.arrow.pyspark.fallback.enabled', 'true'),
        ('spark.dynamicAllocation.enabled', 'false'),
        ('spark.sql.caseSensitive', 'true'),
        ('spark.sql.adaptive.enabled', 'true'),
        ('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension'),
        ('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog'),
        ('spark.sql.warehouse.dir', warehouse_diretory_path),
        ('spark.driver.extraJavaOptions', f'-Dderby.system.home={warehouse_diretory_path}')
    ])

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

In [ ]:
human_methylation_manifest_file = 'file:///[FILE_FULL_NAME:humanmethylation450_15017482_v1-2.csv]'

biological_database_name = 'biological_database'
bronze_human_methylation_manifest_table_name = 'bronze_illumina_human_methylation_450_Manifest'

In [ ]:
human_methylation_manifest_df = spark.read \
    .option('header', True) \
    .options(delimiter=',') \
    .csv(human_methylation_manifest_file)

human_methylation_manifest_df = human_methylation_manifest_df \
    .select([col(f'`{column}`' if '.' in column else column).alias(re.sub('[^0-9a-zA-Z]+', '', column)) for column in human_methylation_manifest_df.columns])

In [ ]:
spark.sql(f'CREATE DATABASE IF NOT EXISTS {biological_database_name};')
spark.sql(f'USE {biological_database_name};')

In [ ]:
human_methylation_manifest_df.write \
    .format('delta') \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .option('partitionOverwriteMode', 'dynamic') \
    .saveAsTable(bronze_human_methylation_manifest_table_name)

In [ ]:
spark.stop()